In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
from ase.build import fcc111, molecule, fcc100, fcc110
from ase.visualize import view
from ase import Atoms
from ase.io import write

In [5]:
from ase.optimize import BFGS, FIRE
from ase.filters import FrechetCellFilter

In [90]:
from amstools.lammps import write_in_lammps

# Pt surface

In [91]:
el='Pt'

In [92]:
# Build an FCC (100) surface
surface = fcc100(el, size=(4, 4, 6), vacuum=20.0)

In [93]:
# Create an O2 molecule
o2 = molecule('O2')
o2.rotate([0,0,1],[1,0,0])

z_max=max(surface.positions[:,2])

# Adjust the position of the O2 molecule above the surface
o2.translate([surface.get_cell()[0, 0] / 2, surface.get_cell()[1, 1] / 2, z_max+2.5])

In [94]:
# Combine the surface and the O2 molecule into a single system
system = surface + o2

# Visualize the system (optional)
# view(system)

In [95]:
fname=f"1-Pt-surface/{el}_surface-fcc-slab_and_O2.lammps-data"
fname

'1-Pt-surface/Pt_surface-fcc-slab_and_O2.lammps-data'

In [97]:


specorder=[el,'O']
specorder

write(fname, system, specorder=specorder)

# Ethanol water

In [62]:
from ase.build import molecule
from ase import Atoms

In [63]:
from matscipy.neighbours import neighbour_list

In [64]:
water=molecule("H2O")

In [65]:
water.get_chemical_symbols()

['O', 'H', 'H']

In [66]:
# water.set_chemical_symbols(['Li','He','He'])

In [67]:
ethanol=molecule("CH3CH2OH")

In [68]:
def insert_molecules(tot_cell, mol, N_mol, MIN_THRESHOLD ):
    
    for _ in range(N_mol):

        while True:
            new_mol=mol.copy()
            new_mol.rotate([0,0,1], np.random.randn(3))

            rnd_shift=np.random.rand(3)*np.diag(tot_cell.cell)

            new_mol.positions+=rnd_shift

            new_tot_cell = tot_cell+new_mol

            min_dist_list=neighbour_list("ijd", new_tot_cell, cutoff=3.0)
            ii,jj,dd=min_dist_list

            new_indices = np.arange(len(new_tot_cell))[-len(new_mol):]
            old_indices = np.arange(len(new_tot_cell))[:-len(new_mol)]

            mask=np.isin(ii, new_indices) & np.isin(jj, old_indices)

            if np.any(mask):
                min_dist = np.min(dd[mask])
            else:
                min_dist = 100


            if min_dist >= MIN_THRESHOLD:
                break

        tot_cell = new_tot_cell                
        print(f"Current structure: num.at={len(tot_cell)}, min_dist={min_dist}, overall min = {np.min(dd)}")        
    return tot_cell


In [69]:
MIN_THRESHOLD=2

In [70]:
tot_cell = ethanol.copy()
tot_cell.set_cell([15,15,15])
tot_cell.set_pbc(True)

In [71]:
tot_cell_1=insert_molecules(tot_cell, ethanol, 10, MIN_THRESHOLD=MIN_THRESHOLD)

Current structure: num.at=18, min_dist=100, overall min = 0.9713238910044372
Current structure: num.at=27, min_dist=100, overall min = 0.9713238910044372
Current structure: num.at=36, min_dist=2.1817103430706766, overall min = 0.9713238910044372
Current structure: num.at=45, min_dist=2.4292042510903205, overall min = 0.9713238910044367
Current structure: num.at=54, min_dist=100, overall min = 0.9713238910044367
Current structure: num.at=63, min_dist=2.9148663735013773, overall min = 0.9713238910044367
Current structure: num.at=72, min_dist=2.7168704228767755, overall min = 0.9713238910044367
Current structure: num.at=81, min_dist=100, overall min = 0.9713238910044367
Current structure: num.at=90, min_dist=2.8734960792302835, overall min = 0.9713238910044367
Current structure: num.at=99, min_dist=2.7421827809672226, overall min = 0.9713238910044367


In [72]:
#view(tot_cell_1)

In [73]:
tot_cell_2=insert_molecules(tot_cell_1, water, 49, MIN_THRESHOLD=MIN_THRESHOLD)

Current structure: num.at=102, min_dist=2.285761035048693, overall min = 0.9685650182625838
Current structure: num.at=105, min_dist=100, overall min = 0.9685650182625838
Current structure: num.at=108, min_dist=2.507345660939723, overall min = 0.9685650182625838
Current structure: num.at=111, min_dist=2.9289748965982256, overall min = 0.9685650182625836
Current structure: num.at=114, min_dist=2.073046026961274, overall min = 0.9685650182625836
Current structure: num.at=117, min_dist=2.434592249465968, overall min = 0.9685650182625836
Current structure: num.at=120, min_dist=100, overall min = 0.9685650182625836
Current structure: num.at=123, min_dist=2.1761948917733553, overall min = 0.9685650182625833
Current structure: num.at=126, min_dist=2.3668248905807294, overall min = 0.9685650182625833
Current structure: num.at=129, min_dist=2.353544351714592, overall min = 0.9685650182625833
Current structure: num.at=132, min_dist=2.1924088187255726, overall min = 0.968565018262583
Current struc

In [74]:
view(tot_cell_2, viewer='x3d')

In [75]:
conv_factor=1.6605402e-27 / 1e-30# amu/A3 -> kg/m3

In [76]:
sum(tot_cell_2.get_masses())/sum(tot_cell_2.get_volume())*conv_factor

np.float64(683.6475984174225)

In [77]:
len(tot_cell_2)

246

In [78]:
tot_cell_2

Atoms(symbols='C22H164O60', pbc=True, cell=[15.0, 15.0, 15.0])

In [79]:
write("2-ethanol-water/ethanol-water.lammps-data", tot_cell_2, format='lammps-data', specorder=['C','H','O'])

In [80]:
min_dist_list=neighbour_list("ijd", tot_cell_1,cutoff=3.0)

In [81]:
np.min(min_dist_list[2])

np.float64(0.9713238910044367)